In [5]:
%pip install spmf

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.3 -> 23.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [6]:
from spmf import Spmf
import pandas as pd
from itertools import chain, combinations

In [7]:
from itertools import chain, combinations

def powerset(s):
    return chain.from_iterable(combinations(s, r) for r in range(1, len(s)))

def associationRule(freqItemSetDict, itemSetList, minConf):
    rules = []

    # Menghitung dukungan setiap itemset dalam freqItemSet
    itemSetSupport = {}
    for itemSet, support in freqItemSetDict:
        itemSetTuple = frozenset(itemSet)  # Menggunakan frozenset sebagai kunci
        itemSetSupport[itemSetTuple] = support

    for itemSet, support in freqItemSetDict:
        subsets = powerset(itemSet)
        itemSetSup = itemSetSupport[frozenset(itemSet)]  # Mengambil dukungan itemSet

        for s in subsets:
            s_tuple = frozenset(s)
            if s_tuple in itemSetSupport and itemSetSupport[s_tuple] > 0:
                confidence = float(itemSetSup / itemSetSupport[s_tuple])
                if confidence > minConf:
                    lift_ratio = confidence / (itemSetSupport[frozenset(itemSet.difference(s))] / len(itemSetList))

                    rules.append([set(s), set(itemSet.difference(s)), support, confidence, lift_ratio])

    return rules


In [8]:
# file_path = '../notebook/data/example_spfm_test.txt'

# file_path = '../notebook/data/transaction_01_sd_03_october_spmf_test.txt'
# minsup: 0.0014, minconf: 0.6, minsuppcount: 2.1 

file_path = '../notebook/data/transaction_01_sd_07_october_spmf_test.txt'
# minsup: 0.0008, minconf: 0.6, minsuppcount: 2.7 

itemSetList = []

with open(file_path, 'r') as file:
    for line in file:
        elements = line.strip().split(' ')
        
        if elements[0].startswith('@'):
            continue  # Lewati baris yang mengandung informasi header
        
        itemSetList.append(elements)


spmf = Spmf("Eclat",
            input_filename=file_path,
            output_filename="../notebook/output/frequent-pattern-eclat.txt",
            arguments=[0.0008, False])
spmf.run()

patterns_list = []
with open("../notebook/output/frequent-pattern-eclat.txt", "r") as file:
    for line in file:
        if line.startswith("#SUP:"):
            continue
        parts = line.strip().split(" #SUP: ")
        pattern = frozenset(map(str, parts[0].split()))
        sup = int(parts[1])
        patterns_list.append((pattern, sup))

# Create association rules
rules = associationRule(patterns_list, itemSetList, minConf=0.7)

# Buat DataFrame
columns = ["Antecedent", "Consequent", "Support", "Confidence", "Lift"]
df = pd.DataFrame(rules, columns=columns)
df_sorted = df.sort_values(by='Confidence', ascending=False)
df_sorted = df_sorted.reset_index(drop=True)
df_sorted

>/D:/dev/project/Skripsi/Skripsi_ECLAT_and_FPGrowth/notebook/spmf.jar
=============  ECLAT v0.96r18 - STATS =============
 Transactions count from database : 3454
 Frequent itemsets count : 1070
 Total time ~ 67 ms
 Maximum memory usage : 24.170013427734375 mb
Post-processing to show result in terms of string values.
Post-processing completed.



,Antecedent,Consequent,Support,Confidence,Lift
0,{8998866200813},{8996006853400},3,1.000000,90.894737
1,{8998866200899},{8998866203272},3,1.000000,431.750000
2,"{089686010107, GLLMP1KG}",{089686010824},5,1.000000,36.357895
3,"{8992770033130, 089686010824, GLLMP1KG}",{089686010107},3,1.000000,43.721519
4,"{8992770033130, 089686010107, GLLMP1KG}",{089686010824},3,1.000000,36.357895
5,"{089686011692, LEMINGLN15}",{089686010107},3,1.000000,43.721519
6,"{711844110113, 089686010824}",{089686010107},3,1.000000,43.721519
7,"{089686021141, 089686021219}",{089686021103},3,1.000000,246.714286
8,{8999999529628},{GLLMP1KG},3,1.000000,54.825397
9,"{089686010824, GLLMP1KG}",{089686010107},5,0.833333,36.434599
